In [ ]:
import numpy as np
if not hasattr(__builtins__, 'object'):
    np.object = object

from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

def load_image_mask_pairs(image_dir, mask_dir, img_size=(256, 256)):
    mask_files = sorted(os.listdir(mask_dir))
    image_files = sorted(os.listdir(image_dir))

    masks = []
    images = []

    for img_name, mask_name in zip(image_files, mask_files):
        mask_path = os.path.join(mask_dir, mask_name)
        img_path = os.path.join(image_dir, img_name)

        image = load_img(img_path, target_size=img_size)
        image = img_to_array(image) / 255.0

        mask = load_img(mask_path, target_size=img_size, color_mode="grayscale")
        mask = img_to_array(mask) / 255.0
        mask = (mask > 0.5).astype(np.float32)

        images.append(image)
        masks.append(mask)

    return np.array(images), np.array(masks)

base_path = os.path.join('C:', os.sep, 'Users', 'USER', 'python', 'pj3', 'ETT-v3', 'Fold1')

train_images, train_masks = load_image_mask_pairs(
    os.path.join(base_path, 'train'),
    os.path.join(base_path, 'trainannot')
)

val_images, val_masks = load_image_mask_pairs(
    os.path.join(base_path, 'val'),
    os.path.join(base_path, 'valannot')
)

test_images, test_masks = load_image_mask_pairs(
    os.path.join(base_path, 'test'),
    os.path.join(base_path, 'testannot')
)

print(len(train_images), len(train_masks))
print(len(val_images), len(val_masks))

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_gen_args = dict(            
    width_shift_range=0.05,     #左右隨機平移
    rotation_range=10,   #輕微旋轉  
    height_shift_range=0.05,    #上下隨機平移            
    zoom_range=[0.9, 1.1],    #隨機裁切與縮放 
    shear_range=0.05,       #剪切變形範圍  
    horizontal_flip=True,    #水平翻轉          
    brightness_range=[0.85, 1.15],   #亮度變化範圍
    fill_mode='nearest'
)

def augment_dataset(images, masks, augment_times=2, batch_size=8):
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    image_gen = image_datagen.flow(images, batch_size=batch_size, seed=1)
    mask_gen = mask_datagen.flow(masks, batch_size=batch_size, seed=1)

    augmented_images = []
    augmented_masks = []

    steps = (len(images) * augment_times) // batch_size

    for _ in range(steps):
        img_batch = next(image_gen)
        mask_batch = next(mask_gen)
        augmented_images.extend(img_batch)
        augmented_masks.extend(mask_batch)

    return np.array(augmented_images), np.array(augmented_masks)


aug_train_images, aug_train_masks = augment_dataset(train_images, train_masks, augment_times=2)

train_images = np.concatenate([train_images, aug_train_images])
train_masks = np.concatenate([train_masks, aug_train_masks])

print((train_images.shape),(test_images.shape))

import tensorflow as tf

def iou(y_true, y_pred):
    intersection = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32))
    union = tf.reduce_sum(tf.cast(y_true + y_pred > 0, tf.float32))
    return intersection / (union + tf.keras.backend.epsilon())

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import LearningRateScheduler
from keras import backend as keras


def fcn_model(input_size=(256, 256, 3)):
    inputs = Input(shape=input_size)

    # Encoder (VGG-like)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)  # 128x128

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)  # 64x64

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)  # 32x32

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)  # 16x16

    conv5 = Conv2D(4096, (7, 7), activation='relu', padding='same')(pool4)
    conv5 = Dropout(0.5)(conv5)
    conv5 = Conv2D(4096, (1, 1), activation='relu', padding='same')(conv5)
    conv5 = Dropout(0.5)(conv5)

    score_fr = Conv2D(1, (1, 1), activation=None, padding='same')(conv5)
    
    # 多段上採樣 (從 16x16 → 256x256)
    x = Conv2DTranspose(512, kernel_size=3, strides=2, padding='same')(score_fr)
    x = Conv2DTranspose(256, kernel_size=3, strides=2, padding='same')(x)
    x = Conv2DTranspose(128, kernel_size=3, strides=2, padding='same')(x)
    x = Conv2DTranspose(64, kernel_size=3, strides=2, padding='same')(x)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy', iou])
    return model


from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

Fold_number = 1

model = fcn_model()
model_checkpoint = ModelCheckpoint(
    f'ETT_fcn_Fold{Fold_number}.keras',
    monitor='val_loss',
    verbose=1,
    save_best_only=True
)

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history_model = model.fit(
    train_images, train_masks,
    batch_size=2,
    epochs=30,
    validation_data=(val_images, val_masks),
    callbacks=[model_checkpoint, early_stop]
)

In [ ]:
import os, numpy as np, tensorflow as tf, cv2
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model

Fold_number = 1
PIXELS_PER_CM = 72

# ========================
# 1. 模型預測與績效計算
# ========================
# 載入測試資料
test_images, test_masks = load_image_mask_pairs(
    os.path.join(f'ETT-v3/Fold{Fold_number}', 'test'),
    os.path.join(f'ETT-v3/Fold{Fold_number}', 'testannot')
)

# 提前準備圖像檔案路徑（用於可視化）
test_image_dir = f'ETT-v3/Fold{Fold_number}/test'
test_image_filenames = sorted(os.listdir(test_image_dir))
test_image_paths = [os.path.join(test_image_dir, name) for name in test_image_filenames]

# 模型預測
model = fcn_model()
model.load_weights(f'ETT_fcn_Fold{Fold_number}.keras')
results = model.predict(test_images, batch_size=8, verbose=1)
results = (results > 0.5).astype(int)

# 儲存預測遮罩
def labelVisualize(num_class, color_dict, img):
    img = img[:, :, 0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i, :] = color_dict[i]
    return img_out / 255

def saveResult(save_path, npyfile, flag_multi_class=False, num_class=2):
    os.makedirs(save_path, exist_ok=True)
    for i, item in enumerate(npyfile):
        img = labelVisualize(num_class, np.array([[0, 0, 0], [255, 255, 255]]), item) if flag_multi_class else item[:, :, 0]
        img = (img * 255).astype(np.uint8)
        img = Image.fromarray(img).convert('RGB')
        img.save(os.path.join(save_path, f"{i}_predict.jpeg"), 'JPEG')

saveResult(f'ETT/Fold{Fold_number}/predannot_fcn/', results)

# 評估指標計算
true_images = test_masks
pred_images = results

def get_lowest_y_coordinate(mask):
    for idx in range(mask.shape[0] - 1, -1, -1):
        if np.any(mask[idx]):
            return idx
    return 0

def extract_lowest_y_coordinates(gt_list, pred_list):
    gt_coords = [get_lowest_y_coordinate(gt) for gt in gt_list]
    pred_coords = [get_lowest_y_coordinate(pd) for pd in pred_list]
    return tf.constant(gt_coords, dtype=tf.float32), tf.constant(pred_coords, dtype=tf.float32)

def compute_iou(gt_mask, pred_mask):
    metric = tf.keras.metrics.MeanIoU(num_classes=2)
    metric.update_state(gt_mask, pred_mask)
    return metric.result().numpy()

def mean_vertical_error(gt_list, pred_list):
    actual_y, predicted_y = extract_lowest_y_coordinates(gt_list, pred_list)
    return tf.reduce_mean(tf.abs(predicted_y - actual_y)) / PIXELS_PER_CM

def precision_within_threshold(gt_list, pred_list, threshold):
    actual_y, predicted_y = extract_lowest_y_coordinates(gt_list, pred_list)
    acc = tf.reduce_mean(tf.cast(tf.abs(predicted_y - actual_y) <= threshold, tf.float32)) * 100.0
    return acc.numpy()

def precision_within_half_cm(gt, pd): return precision_within_threshold(gt, pd, 36)
def precision_within_one_cm(gt, pd): return precision_within_threshold(gt, pd, 72)

print("📊 Evaluation Results")
print(f"IoU: {compute_iou(true_images, pred_images):.4f}")
print(f"Avg Vertical Error (cm): {mean_vertical_error(true_images, pred_images).numpy():.4f}")
print(f"Acc ≤ 0.5 cm: {precision_within_half_cm(true_images, pred_images):.2f}%")
print(f"Acc ≤ 1.0 cm: {precision_within_one_cm(true_images, pred_images):.2f}%")

# ========================
# 2. 圖像可視化（未標記端點）
# ========================
def preprocess_for_display(mask_array, cmap='plasma'):
    plt.imshow(mask_array.squeeze(), cmap=cmap)
    plt.axis('off')

i = 0  # 預覽哪張圖

plt.figure(figsize=(9, 2.5))
plt.subplot(131)
plt.title("Image")
plt.imshow(Image.open(test_image_paths[i]), cmap='gray')
plt.axis('off')

plt.subplot(132)
plt.title("Ground Truth Mask")
preprocess_for_display(test_masks[i])

plt.subplot(133)
plt.title("Predicted Mask")
preprocess_for_display(results[i])

plt.tight_layout()
plt.show()

# ========================
# 3. 圖像可視化（已標記端點）
# ========================
def show_image_with_endpoint(mask_array, title, label=None, color='red'):
    img = mask_array.squeeze()
    y = get_lowest_y_coordinate(img)
    plt.imshow(img, cmap='plasma')
    if label:
        plt.scatter(img.shape[1] // 2, y, c=color, s=30)
        plt.text(img.shape[1] // 2 + 5, y, label, color=color,
                 fontsize=12, fontweight='bold', va='center')
    plt.title(title)
    plt.axis('off')

plt.figure(figsize=(9, 2.5))
plt.subplot(131)
plt.imshow(Image.open(test_image_paths[i]), cmap='gray')
plt.title('Image')
plt.axis('off')

plt.subplot(132)
show_image_with_endpoint(test_masks[i], "Ground Truth Mask with Endpoint", label="G", color='red')

plt.subplot(133)
show_image_with_endpoint(results[i], "Predicted Mask with Endpoint", label="Y", color='yellow')

plt.tight_layout()
plt.show()


In [ ]:
import gc
from tensorflow.keras import backend as K

del model
K.clear_session()
gc.collect()
print(f"🧹 Fold{Fold_number} 記憶體已釋放")


In [ ]:
# ========== ⬇ Fold2 的 Cell 開頭，清除上一輪模型記憶體 ==========
from tensorflow.keras import backend as K
import gc
K.clear_session()
gc.collect()

# ========== ⬇ 載入 Fold2 的資料集 ==========
Fold_number = 2  # <--- 改成你這次要跑的 Fold 編號

base_path = os.path.join('C:', os.sep, 'Users', 'USER', 'python', 'pj3', 'ETT-v3', f'Fold{Fold_number}')
train_images, train_masks = load_image_mask_pairs(
    os.path.join(base_path, 'train'),
    os.path.join(base_path, 'trainannot')
)
val_images, val_masks = load_image_mask_pairs(
    os.path.join(base_path, 'val'),
    os.path.join(base_path, 'valannot')
)

# 資料增強（如需跳過可註解）
aug_train_images, aug_train_masks = augment_dataset(train_images, train_masks, augment_times=2)
train_images = np.concatenate([train_images, aug_train_images])
train_masks = np.concatenate([train_masks, aug_train_masks])

print(f"✅ Fold{Fold_number} loaded:", train_images.shape, val_images.shape)

# ========== ⬇ 建立與訓練模型 ==========
model = fcn_model()
model_checkpoint = ModelCheckpoint(
    f'ETT_fcn_Fold{Fold_number}.keras',
    monitor='val_loss',
    verbose=1,
    save_best_only=True
)
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history_model = model.fit(
    train_images, train_masks,
    batch_size=4,
    epochs=10,
    validation_data=(val_images, val_masks),
    callbacks=[model_checkpoint, early_stop]
)


In [ ]:
import os, numpy as np, tensorflow as tf, cv2
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model

Fold_number = 2
PIXELS_PER_CM = 72

# ========================
# 1. 模型預測與績效計算
# ========================
# 載入測試資料
test_images, test_masks = load_image_mask_pairs(
    os.path.join(f'ETT-v3/Fold{Fold_number}', 'test'),
    os.path.join(f'ETT-v3/Fold{Fold_number}', 'testannot')
)

# 提前準備圖像檔案路徑（用於可視化）
test_image_dir = f'ETT-v3/Fold{Fold_number}/test'
test_image_filenames = sorted(os.listdir(test_image_dir))
test_image_paths = [os.path.join(test_image_dir, name) for name in test_image_filenames]

# 模型預測
model = fcn_model()
model.load_weights(f'ETT_fcn_Fold{Fold_number}.keras')
results = model.predict(test_images, batch_size=8, verbose=1)
results = (results > 0.5).astype(int)

# 儲存預測遮罩
def labelVisualize(num_class, color_dict, img):
    img = img[:, :, 0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i, :] = color_dict[i]
    return img_out / 255

def saveResult(save_path, npyfile, flag_multi_class=False, num_class=2):
    os.makedirs(save_path, exist_ok=True)
    for i, item in enumerate(npyfile):
        img = labelVisualize(num_class, np.array([[0, 0, 0], [255, 255, 255]]), item) if flag_multi_class else item[:, :, 0]
        img = (img * 255).astype(np.uint8)
        img = Image.fromarray(img).convert('RGB')
        img.save(os.path.join(save_path, f"{i}_predict.jpeg"), 'JPEG')

saveResult(f'ETT/Fold{Fold_number}/predannot_fcn/', results)

# 評估指標計算
true_images = test_masks
pred_images = results

def get_lowest_y_coordinate(mask):
    for idx in range(mask.shape[0] - 1, -1, -1):
        if np.any(mask[idx]):
            return idx
    return 0

def extract_lowest_y_coordinates(gt_list, pred_list):
    gt_coords = [get_lowest_y_coordinate(gt) for gt in gt_list]
    pred_coords = [get_lowest_y_coordinate(pd) for pd in pred_list]
    return tf.constant(gt_coords, dtype=tf.float32), tf.constant(pred_coords, dtype=tf.float32)

def compute_iou(gt_mask, pred_mask):
    metric = tf.keras.metrics.MeanIoU(num_classes=2)
    metric.update_state(gt_mask, pred_mask)
    return metric.result().numpy()

def mean_vertical_error(gt_list, pred_list):
    actual_y, predicted_y = extract_lowest_y_coordinates(gt_list, pred_list)
    return tf.reduce_mean(tf.abs(predicted_y - actual_y)) / PIXELS_PER_CM

def precision_within_threshold(gt_list, pred_list, threshold):
    actual_y, predicted_y = extract_lowest_y_coordinates(gt_list, pred_list)
    acc = tf.reduce_mean(tf.cast(tf.abs(predicted_y - actual_y) <= threshold, tf.float32)) * 100.0
    return acc.numpy()

def precision_within_half_cm(gt, pd): return precision_within_threshold(gt, pd, 36)
def precision_within_one_cm(gt, pd): return precision_within_threshold(gt, pd, 72)

print("📊 Evaluation Results")
print(f"IoU: {compute_iou(true_images, pred_images):.4f}")
print(f"Avg Vertical Error (cm): {mean_vertical_error(true_images, pred_images).numpy():.4f}")
print(f"Acc ≤ 0.5 cm: {precision_within_half_cm(true_images, pred_images):.2f}%")
print(f"Acc ≤ 1.0 cm: {precision_within_one_cm(true_images, pred_images):.2f}%")

# ========================
# 2. 圖像可視化（未標記端點）
# ========================
def preprocess_for_display(mask_array, cmap='plasma'):
    plt.imshow(mask_array.squeeze(), cmap=cmap)
    plt.axis('off')

i = 0  # 預覽哪張圖

plt.figure(figsize=(9, 2.5))
plt.subplot(131)
plt.title("Image")
plt.imshow(Image.open(test_image_paths[i]), cmap='gray')
plt.axis('off')

plt.subplot(132)
plt.title("Ground Truth Mask")
preprocess_for_display(test_masks[i])

plt.subplot(133)
plt.title("Predicted Mask")
preprocess_for_display(results[i])

plt.tight_layout()
plt.show()

# ========================
# 3. 圖像可視化（已標記端點）
# ========================
def show_image_with_endpoint(mask_array, title, label=None, color='red'):
    img = mask_array.squeeze()
    y = get_lowest_y_coordinate(img)
    plt.imshow(img, cmap='plasma')
    if label:
        plt.scatter(img.shape[1] // 2, y, c=color, s=30)
        plt.text(img.shape[1] // 2 + 5, y, label, color=color,
                 fontsize=12, fontweight='bold', va='center')
    plt.title(title)
    plt.axis('off')

plt.figure(figsize=(9, 2.5))
plt.subplot(131)
plt.imshow(Image.open(test_image_paths[i]), cmap='gray')
plt.title('Image')
plt.axis('off')

plt.subplot(132)
show_image_with_endpoint(test_masks[i], "Ground Truth Mask with Endpoint", label="G", color='red')

plt.subplot(133)
show_image_with_endpoint(results[i], "Predicted Mask with Endpoint", label="Y", color='yellow')

plt.tight_layout()
plt.show()


In [ ]:
import gc
from tensorflow.keras import backend as K

del model
K.clear_session()
gc.collect()
print(f"🧹 Fold{Fold_number} 記憶體已釋放")


In [ ]:
from tensorflow.keras import backend as K
import gc
K.clear_session()
gc.collect()

# ========== 載入資料集 ==========
Fold_number = 3  # <--- 改成你這次要跑的 Fold 編號

base_path = os.path.join('C:', os.sep, 'Users', 'USER', 'python', 'pj3', 'ETT-v3', f'Fold{Fold_number}')
train_images, train_masks = load_image_mask_pairs(
    os.path.join(base_path, 'train'),
    os.path.join(base_path, 'trainannot')
)
val_images, val_masks = load_image_mask_pairs(
    os.path.join(base_path, 'val'),
    os.path.join(base_path, 'valannot')
)

# 資料增強
aug_train_images, aug_train_masks = augment_dataset(train_images, train_masks, augment_times=2)
train_images = np.concatenate([train_images, aug_train_images])
train_masks = np.concatenate([train_masks, aug_train_masks])

print(f"✅ Fold{Fold_number} loaded:", train_images.shape, val_images.shape)

# ========== ⬇ 建立與訓練模型 ==========
model = fcn_model()
model_checkpoint = ModelCheckpoint(
    f'ETT_fcn_Fold{Fold_number}.keras',
    monitor='val_loss',
    verbose=1,
    save_best_only=True
)
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history_model = model.fit(
    train_images, train_masks,
    batch_size=4,
    epochs=10,
    validation_data=(val_images, val_masks),
    callbacks=[model_checkpoint, early_stop]
)


In [ ]:
import os, numpy as np, tensorflow as tf, cv2
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model

Fold_number = 3
PIXELS_PER_CM = 72

# ========================
# 1. 模型預測與績效計算
# ========================
# 載入測試資料
test_images, test_masks = load_image_mask_pairs(
    os.path.join(f'ETT-v3/Fold{Fold_number}', 'test'),
    os.path.join(f'ETT-v3/Fold{Fold_number}', 'testannot')
)

# 提前準備圖像檔案路徑（用於可視化）
test_image_dir = f'ETT-v3/Fold{Fold_number}/test'
test_image_filenames = sorted(os.listdir(test_image_dir))
test_image_paths = [os.path.join(test_image_dir, name) for name in test_image_filenames]

# 模型預測
model = fcn_model()
model.load_weights(f'ETT_fcn_Fold{Fold_number}.keras')
results = model.predict(test_images, batch_size=8, verbose=1)
results = (results > 0.5).astype(int)

# 儲存預測遮罩
def labelVisualize(num_class, color_dict, img):
    img = img[:, :, 0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i, :] = color_dict[i]
    return img_out / 255

def saveResult(save_path, npyfile, flag_multi_class=False, num_class=2):
    os.makedirs(save_path, exist_ok=True)
    for i, item in enumerate(npyfile):
        img = labelVisualize(num_class, np.array([[0, 0, 0], [255, 255, 255]]), item) if flag_multi_class else item[:, :, 0]
        img = (img * 255).astype(np.uint8)
        img = Image.fromarray(img).convert('RGB')
        img.save(os.path.join(save_path, f"{i}_predict.jpeg"), 'JPEG')

saveResult(f'ETT/Fold{Fold_number}/predannot_fcn/', results)

# 評估指標計算
true_images = test_masks
pred_images = results

def get_lowest_y_coordinate(mask):
    for idx in range(mask.shape[0] - 1, -1, -1):
        if np.any(mask[idx]):
            return idx
    return 0

def extract_lowest_y_coordinates(gt_list, pred_list):
    gt_coords = [get_lowest_y_coordinate(gt) for gt in gt_list]
    pred_coords = [get_lowest_y_coordinate(pd) for pd in pred_list]
    return tf.constant(gt_coords, dtype=tf.float32), tf.constant(pred_coords, dtype=tf.float32)

def compute_iou(gt_mask, pred_mask):
    metric = tf.keras.metrics.MeanIoU(num_classes=2)
    metric.update_state(gt_mask, pred_mask)
    return metric.result().numpy()

def mean_vertical_error(gt_list, pred_list):
    actual_y, predicted_y = extract_lowest_y_coordinates(gt_list, pred_list)
    return tf.reduce_mean(tf.abs(predicted_y - actual_y)) / PIXELS_PER_CM

def precision_within_threshold(gt_list, pred_list, threshold):
    actual_y, predicted_y = extract_lowest_y_coordinates(gt_list, pred_list)
    acc = tf.reduce_mean(tf.cast(tf.abs(predicted_y - actual_y) <= threshold, tf.float32)) * 100.0
    return acc.numpy()

def precision_within_half_cm(gt, pd): return precision_within_threshold(gt, pd, 36)
def precision_within_one_cm(gt, pd): return precision_within_threshold(gt, pd, 72)

print("📊 Evaluation Results")
print(f"IoU: {compute_iou(true_images, pred_images):.4f}")
print(f"Avg Vertical Error (cm): {mean_vertical_error(true_images, pred_images).numpy():.4f}")
print(f"Acc ≤ 0.5 cm: {precision_within_half_cm(true_images, pred_images):.2f}%")
print(f"Acc ≤ 1.0 cm: {precision_within_one_cm(true_images, pred_images):.2f}%")

# ========================
# 2. 圖像可視化（未標記端點）
# ========================
def preprocess_for_display(mask_array, cmap='plasma'):
    plt.imshow(mask_array.squeeze(), cmap=cmap)
    plt.axis('off')

i = 0  # 預覽哪張圖

plt.figure(figsize=(9, 2.5))
plt.subplot(131)
plt.title("Image")
plt.imshow(Image.open(test_image_paths[i]), cmap='gray')
plt.axis('off')

plt.subplot(132)
plt.title("Ground Truth Mask")
preprocess_for_display(test_masks[i])

plt.subplot(133)
plt.title("Predicted Mask")
preprocess_for_display(results[i])

plt.tight_layout()
plt.show()

# ========================
# 3. 圖像可視化（已標記端點）
# ========================
def show_image_with_endpoint(mask_array, title, label=None, color='red'):
    img = mask_array.squeeze()
    y = get_lowest_y_coordinate(img)
    plt.imshow(img, cmap='plasma')
    if label:
        plt.scatter(img.shape[1] // 2, y, c=color, s=30)
        plt.text(img.shape[1] // 2 + 5, y, label, color=color,
                 fontsize=12, fontweight='bold', va='center')
    plt.title(title)
    plt.axis('off')

plt.figure(figsize=(9, 2.5))
plt.subplot(131)
plt.imshow(Image.open(test_image_paths[i]), cmap='gray')
plt.title('Image')
plt.axis('off')

plt.subplot(132)
show_image_with_endpoint(test_masks[i], "Ground Truth Mask with Endpoint", label="G", color='red')

plt.subplot(133)
show_image_with_endpoint(results[i], "Predicted Mask with Endpoint", label="Y", color='yellow')

plt.tight_layout()
plt.show()


In [ ]:
import gc
from tensorflow.keras import backend as K

del model
K.clear_session()
gc.collect()
print(f"🧹 Fold{Fold_number} 記憶體已釋放")


In [ ]:
from tensorflow.keras import backend as K
import gc
K.clear_session()
gc.collect()

# ========== 載入資料集 ==========
Fold_number = 4  # <--- 改成你這次要跑的 Fold 編號

base_path = os.path.join('C:', os.sep, 'Users', 'USER', 'python', 'pj3', 'ETT-v3', f'Fold{Fold_number}')
train_images, train_masks = load_image_mask_pairs(
    os.path.join(base_path, 'train'),
    os.path.join(base_path, 'trainannot')
)
val_images, val_masks = load_image_mask_pairs(
    os.path.join(base_path, 'val'),
    os.path.join(base_path, 'valannot')
)

# 資料增強
aug_train_images, aug_train_masks = augment_dataset(train_images, train_masks, augment_times=2)
train_images = np.concatenate([train_images, aug_train_images])
train_masks = np.concatenate([train_masks, aug_train_masks])

print(f"✅ Fold{Fold_number} loaded:", train_images.shape, val_images.shape)

# ========== ⬇ 建立與訓練模型 ==========
model = fcn_model()
model_checkpoint = ModelCheckpoint(
    f'ETT_fcn_Fold{Fold_number}.keras',
    monitor='val_loss',
    verbose=1,
    save_best_only=True
)
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history_model = model.fit(
    train_images, train_masks,
    batch_size=4,
    epochs=10,
    validation_data=(val_images, val_masks),
    callbacks=[model_checkpoint, early_stop]
)


In [ ]:
import os, numpy as np, tensorflow as tf, cv2
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model

Fold_number = 4
PIXELS_PER_CM = 72

# ========================
# 1. 模型預測與績效計算
# ========================
# 載入測試資料
test_images, test_masks = load_image_mask_pairs(
    os.path.join(f'ETT-v3/Fold{Fold_number}', 'test'),
    os.path.join(f'ETT-v3/Fold{Fold_number}', 'testannot')
)

# 提前準備圖像檔案路徑（用於可視化）
test_image_dir = f'ETT-v3/Fold{Fold_number}/test'
test_image_filenames = sorted(os.listdir(test_image_dir))
test_image_paths = [os.path.join(test_image_dir, name) for name in test_image_filenames]

# 模型預測
model = fcn_model()
model.load_weights(f'ETT_fcn_Fold{Fold_number}.keras')
results = model.predict(test_images, batch_size=8, verbose=1)
results = (results > 0.5).astype(int)

# 儲存預測遮罩
def labelVisualize(num_class, color_dict, img):
    img = img[:, :, 0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i, :] = color_dict[i]
    return img_out / 255

def saveResult(save_path, npyfile, flag_multi_class=False, num_class=2):
    os.makedirs(save_path, exist_ok=True)
    for i, item in enumerate(npyfile):
        img = labelVisualize(num_class, np.array([[0, 0, 0], [255, 255, 255]]), item) if flag_multi_class else item[:, :, 0]
        img = (img * 255).astype(np.uint8)
        img = Image.fromarray(img).convert('RGB')
        img.save(os.path.join(save_path, f"{i}_predict.jpeg"), 'JPEG')

saveResult(f'ETT/Fold{Fold_number}/predannot_fcn/', results)

# 評估指標計算
true_images = test_masks
pred_images = results

def get_lowest_y_coordinate(mask):
    for idx in range(mask.shape[0] - 1, -1, -1):
        if np.any(mask[idx]):
            return idx
    return 0

def extract_lowest_y_coordinates(gt_list, pred_list):
    gt_coords = [get_lowest_y_coordinate(gt) for gt in gt_list]
    pred_coords = [get_lowest_y_coordinate(pd) for pd in pred_list]
    return tf.constant(gt_coords, dtype=tf.float32), tf.constant(pred_coords, dtype=tf.float32)

def compute_iou(gt_mask, pred_mask):
    metric = tf.keras.metrics.MeanIoU(num_classes=2)
    metric.update_state(gt_mask, pred_mask)
    return metric.result().numpy()

def mean_vertical_error(gt_list, pred_list):
    actual_y, predicted_y = extract_lowest_y_coordinates(gt_list, pred_list)
    return tf.reduce_mean(tf.abs(predicted_y - actual_y)) / PIXELS_PER_CM

def precision_within_threshold(gt_list, pred_list, threshold):
    actual_y, predicted_y = extract_lowest_y_coordinates(gt_list, pred_list)
    acc = tf.reduce_mean(tf.cast(tf.abs(predicted_y - actual_y) <= threshold, tf.float32)) * 100.0
    return acc.numpy()

def precision_within_half_cm(gt, pd): return precision_within_threshold(gt, pd, 36)
def precision_within_one_cm(gt, pd): return precision_within_threshold(gt, pd, 72)

print("📊 Evaluation Results")
print(f"IoU: {compute_iou(true_images, pred_images):.4f}")
print(f"Avg Vertical Error (cm): {mean_vertical_error(true_images, pred_images).numpy():.4f}")
print(f"Acc ≤ 0.5 cm: {precision_within_half_cm(true_images, pred_images):.2f}%")
print(f"Acc ≤ 1.0 cm: {precision_within_one_cm(true_images, pred_images):.2f}%")

# ========================
# 2. 圖像可視化（未標記端點）
# ========================
def preprocess_for_display(mask_array, cmap='plasma'):
    plt.imshow(mask_array.squeeze(), cmap=cmap)
    plt.axis('off')

i = 0  # 預覽哪張圖

plt.figure(figsize=(9, 2.5))
plt.subplot(131)
plt.title("Image")
plt.imshow(Image.open(test_image_paths[i]), cmap='gray')
plt.axis('off')

plt.subplot(132)
plt.title("Ground Truth Mask")
preprocess_for_display(test_masks[i])

plt.subplot(133)
plt.title("Predicted Mask")
preprocess_for_display(results[i])

plt.tight_layout()
plt.show()

# ========================
# 3. 圖像可視化（已標記端點）
# ========================
def show_image_with_endpoint(mask_array, title, label=None, color='red'):
    img = mask_array.squeeze()
    y = get_lowest_y_coordinate(img)
    plt.imshow(img, cmap='plasma')
    if label:
        plt.scatter(img.shape[1] // 2, y, c=color, s=30)
        plt.text(img.shape[1] // 2 + 5, y, label, color=color,
                 fontsize=12, fontweight='bold', va='center')
    plt.title(title)
    plt.axis('off')

plt.figure(figsize=(9, 2.5))
plt.subplot(131)
plt.imshow(Image.open(test_image_paths[i]), cmap='gray')
plt.title('Image')
plt.axis('off')

plt.subplot(132)
show_image_with_endpoint(test_masks[i], "Ground Truth Mask with Endpoint", label="G", color='red')

plt.subplot(133)
show_image_with_endpoint(results[i], "Predicted Mask with Endpoint", label="Y", color='yellow')

plt.tight_layout()
plt.show()


In [ ]:
import gc
from tensorflow.keras import backend as K

del model
K.clear_session()
gc.collect()
print(f"🧹 Fold{Fold_number} 記憶體已釋放")

In [ ]:
from tensorflow.keras import backend as K
import gc
K.clear_session()
gc.collect()

# ========== 載入資料集 ==========
Fold_number = 5  # <--- 改成你這次要跑的 Fold 編號

base_path = os.path.join('C:', os.sep, 'Users', 'USER', 'python', 'pj3', 'ETT-v3', f'Fold{Fold_number}')
train_images, train_masks = load_image_mask_pairs(
    os.path.join(base_path, 'train'),
    os.path.join(base_path, 'trainannot')
)
val_images, val_masks = load_image_mask_pairs(
    os.path.join(base_path, 'val'),
    os.path.join(base_path, 'valannot')
)

# 資料增強
aug_train_images, aug_train_masks = augment_dataset(train_images, train_masks, augment_times=2)
train_images = np.concatenate([train_images, aug_train_images])
train_masks = np.concatenate([train_masks, aug_train_masks])

print(f"✅ Fold{Fold_number} loaded:", train_images.shape, val_images.shape)

# ========== ⬇ 建立與訓練模型 ==========
model = fcn_model()
model_checkpoint = ModelCheckpoint(
    f'ETT_fcn_Fold{Fold_number}.keras',
    monitor='val_loss',
    verbose=1,
    save_best_only=True
)
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history_model = model.fit(
    train_images, train_masks,
    batch_size=4,
    epochs=10,
    validation_data=(val_images, val_masks),
    callbacks=[model_checkpoint, early_stop]
)


In [ ]:
import os, numpy as np, tensorflow as tf, cv2
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model

Fold_number = 5
PIXELS_PER_CM = 72

# ========================
# 1. 模型預測與績效計算
# ========================
# 載入測試資料
test_images, test_masks = load_image_mask_pairs(
    os.path.join(f'ETT-v3/Fold{Fold_number}', 'test'),
    os.path.join(f'ETT-v3/Fold{Fold_number}', 'testannot')
)

# 提前準備圖像檔案路徑（用於可視化）
test_image_dir = f'ETT-v3/Fold{Fold_number}/test'
test_image_filenames = sorted(os.listdir(test_image_dir))
test_image_paths = [os.path.join(test_image_dir, name) for name in test_image_filenames]

# 模型預測
model = fcn_model()
model.load_weights(f'ETT_fcn_Fold{Fold_number}.keras')
results = model.predict(test_images, batch_size=8, verbose=1)
results = (results > 0.5).astype(int)

# 儲存預測遮罩
def labelVisualize(num_class, color_dict, img):
    img = img[:, :, 0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i, :] = color_dict[i]
    return img_out / 255

def saveResult(save_path, npyfile, flag_multi_class=False, num_class=2):
    os.makedirs(save_path, exist_ok=True)
    for i, item in enumerate(npyfile):
        img = labelVisualize(num_class, np.array([[0, 0, 0], [255, 255, 255]]), item) if flag_multi_class else item[:, :, 0]
        img = (img * 255).astype(np.uint8)
        img = Image.fromarray(img).convert('RGB')
        img.save(os.path.join(save_path, f"{i}_predict.jpeg"), 'JPEG')

saveResult(f'ETT/Fold{Fold_number}/predannot_fcn/', results)

# 評估指標計算
true_images = test_masks
pred_images = results

def get_lowest_y_coordinate(mask):
    for idx in range(mask.shape[0] - 1, -1, -1):
        if np.any(mask[idx]):
            return idx
    return 0

def extract_lowest_y_coordinates(gt_list, pred_list):
    gt_coords = [get_lowest_y_coordinate(gt) for gt in gt_list]
    pred_coords = [get_lowest_y_coordinate(pd) for pd in pred_list]
    return tf.constant(gt_coords, dtype=tf.float32), tf.constant(pred_coords, dtype=tf.float32)

def compute_iou(gt_mask, pred_mask):
    metric = tf.keras.metrics.MeanIoU(num_classes=2)
    metric.update_state(gt_mask, pred_mask)
    return metric.result().numpy()

def mean_vertical_error(gt_list, pred_list):
    actual_y, predicted_y = extract_lowest_y_coordinates(gt_list, pred_list)
    return tf.reduce_mean(tf.abs(predicted_y - actual_y)) / PIXELS_PER_CM

def precision_within_threshold(gt_list, pred_list, threshold):
    actual_y, predicted_y = extract_lowest_y_coordinates(gt_list, pred_list)
    acc = tf.reduce_mean(tf.cast(tf.abs(predicted_y - actual_y) <= threshold, tf.float32)) * 100.0
    return acc.numpy()

def precision_within_half_cm(gt, pd): return precision_within_threshold(gt, pd, 36)
def precision_within_one_cm(gt, pd): return precision_within_threshold(gt, pd, 72)

print("📊 Evaluation Results")
print(f"IoU: {compute_iou(true_images, pred_images):.4f}")
print(f"Avg Vertical Error (cm): {mean_vertical_error(true_images, pred_images).numpy():.4f}")
print(f"Acc ≤ 0.5 cm: {precision_within_half_cm(true_images, pred_images):.2f}%")
print(f"Acc ≤ 1.0 cm: {precision_within_one_cm(true_images, pred_images):.2f}%")

# ========================
# 2. 圖像可視化（未標記端點）
# ========================
def preprocess_for_display(mask_array, cmap='plasma'):
    plt.imshow(mask_array.squeeze(), cmap=cmap)
    plt.axis('off')

i = 0  # 預覽哪張圖

plt.figure(figsize=(9, 2.5))
plt.subplot(131)
plt.title("Image")
plt.imshow(Image.open(test_image_paths[i]), cmap='gray')
plt.axis('off')

plt.subplot(132)
plt.title("Ground Truth Mask")
preprocess_for_display(test_masks[i])

plt.subplot(133)
plt.title("Predicted Mask")
preprocess_for_display(results[i])

plt.tight_layout()
plt.show()

# ========================
# 3. 圖像可視化（已標記端點）
# ========================
def show_image_with_endpoint(mask_array, title, label=None, color='red'):
    img = mask_array.squeeze()
    y = get_lowest_y_coordinate(img)
    plt.imshow(img, cmap='plasma')
    if label:
        plt.scatter(img.shape[1] // 2, y, c=color, s=30)
        plt.text(img.shape[1] // 2 + 5, y, label, color=color,
                 fontsize=12, fontweight='bold', va='center')
    plt.title(title)
    plt.axis('off')

plt.figure(figsize=(9, 2.5))
plt.subplot(131)
plt.imshow(Image.open(test_image_paths[i]), cmap='gray')
plt.title('Image')
plt.axis('off')

plt.subplot(132)
show_image_with_endpoint(test_masks[i], "Ground Truth Mask with Endpoint", label="G", color='red')

plt.subplot(133)
show_image_with_endpoint(results[i], "Predicted Mask with Endpoint", label="Y", color='yellow')

plt.tight_layout()
plt.show()


In [ ]:
import gc
from tensorflow.keras import backend as K

del model
K.clear_session()
gc.collect()
print(f"🧹 Fold{Fold_number} 記憶體已釋放")